# 实验六：爬虫与正则表达式

In [1]:
import numpy as np         
import pandas as pd        
import time                 
import requests            
import re                   

## 正则表达式

In [2]:
p1 = r'<tr.+?ter">.+?(\d{4}-\d{2}-\d{2}).+?</div.+?ter">(.+?)</div.+?ter">(.+?)</div.+?ter">(.+?)</div.+?ter">(.+?)</div.+?ter">(.+?)</div.+?ter">(.+?)</div>'

## 正则表达式分解说明

### 初始部分：`<tr.+?ter">`
- `<tr` - 匹配HTML表格行的开始标签
- `.+?` - 非贪婪模式匹配任意字符（包括换行符，因为使用了re.S标志），直到遇到下一个指定的模式
- `ter">` - 匹配某种属性值的结尾部分，可能是"filter"或其他以"ter"结尾的属性

这部分目的是找到表格行的开始并跳过HTML标签的属性部分。

### 日期提取：`.+?(\d{4}-\d{2}-\d{2})`
- `.+?` - 继续非贪婪匹配任意字符
- `(\d{4}-\d{2}-\d{2})` - 第一个捕获组，匹配日期格式：
  - `\d{4}` - 四位数字（年份，如"2023"）
  - `-` - 连字符
  - `\d{2}` - 两位数字（月份，如"05"）
  - `-` - 连字符
  - `\d{2}` - 两位数字（日期，如"21"）

这部分用于捕获股票数据的交易日期，格式为"YYYY-MM-DD"。

### 重复的数据字段提取模式：`.+?</div.+?ter">(.+?)</div`
这个模式在正则表达式中重复了6次，用于提取表格中的6个不同数据字段（可能包括开盘价、收盘价、最高价、最低价、成交量、成交额等）。

每个重复部分的解析：
- `.+?</div` - 匹配任意字符，直到遇到HTML标签`</div>`的结束
- `.+?ter">` - 匹配任意字符，直到遇到字符串`ter">`
- `(.+?)` - 捕获组，非贪婪匹配任意字符（这里是实际的数据值）
- `</div` - 匹配HTML结束标签`</div>`

这种模式使用非贪婪匹配（`+?`）确保提取的是每个数据单元格的确切内容，不会过度匹配到其他单元格。

## 工作原理举例

假设有这样的HTML片段：
```html
<tr class="filter">
  <td class="filter"><div class="filter">2022-05-15</div></td>
  <td class="filter"><div class="filter">25.30</div></td>
  <td class="filter"><div class="filter">25.64</div></td>
  <td class="filter"><div class="filter">25.12</div></td>
  <td class="filter"><div class="filter">25.45</div></td>
  <td class="filter"><div class="filter">86753</div></td>
  <td class="filter"><div class="filter">2209895</div></td>
</tr>
```

应用此正则表达式后，会捕获7个组：
1. 日期: `2022-05-15`
2. 第一个数值: `25.30` (可能是开盘价)
3. 第二个数值: `25.64` (可能是最高价)
4. 第三个数值: `25.12` (可能是最低价)
5. 第四个数值: `25.45` (可能是收盘价)
6. 第五个数值: `86753` (可能是成交量)
7. 第六个数值: `2209895` (可能是成交额)

## 技巧与注意事项

1. **非贪婪匹配**：表达式中多次使用了`.+?`而不是`.+`，这是非贪婪匹配，确保只匹配到最近的目标模式而不会过度匹配。

2. **捕获组**：表达式中有7个圆括号对`()`，每个都是一个捕获组，对应代码中的`line[0]`到`line[6]`。

3. **re.S标志**：代码中使用了`re.S`标志，使得`.`能匹配包括换行符在内的所有字符，这对于处理可能跨多行的HTML非常重要。

4. **特定HTML结构依赖**：这个正则表达式是针对特定HTML结构设计的，如果网页结构发生变化，正则表达式可能需要调整。

通过这个复杂的正则表达式，代码能够从HTML文件中准确提取出股票的日期和相关交易数据，然后将其保存到CSV文件中供后续分析使用。

In [3]:
fw = open('data.csv', 'w')

data_path = '../../实验六/assets/data/Data_600618/'

# 循环遍历1999年的数据（注意这里只处理了1999年一年的数据）
for nian in range(1999, 2000):
    # 循环遍历每个季度的数据（1到4季度）
    for jidu in range(1, 5):
        # 打开对应年份和季度的HTML文本文件
        with open(data_path + 'DataHTML_600618_Year_' + str(nian) + '_Jidu_' + str(jidu) + '.txt', 'r', encoding='utf-8') as file:
            # 读取HTML内容
            html = file.read()
            # 使用正则表达式查找所有匹配的数据
            match = re.findall(p1, html, re.S)  # re.S使.能匹配包括换行符在内的所有字符
            # 如果找到匹配项
            if match:
                # 遍历每一行匹配的数据
                for line in match:
                    # 将数据写入CSV文件，以逗号分隔
                    fw.write('{:s}, {:s}, {:s}, {:s}, {:s}, {:s}, {:s}\n'.format(line[0], line[1], line[2], line[3],
                                                                                 line[4], line[5], line[6]))
# 关闭文件
fw.close()

## 爬虫

## 第一个正则表达式：pcheck

```python
pcheck = r'<div\sclass="hs01">.+?<div\sclass="hs01">'
```

这个正则表达式的目的是从网页中提取包含新闻列表的整个区块。让我来分解它的各个部分：

### 1. `<div\sclass="hs01">`
- `<div` - 匹配HTML的div标签的开始
- `\s` - 匹配一个空白字符（空格、制表符等）
- `class="hs01"` - 匹配具有"hs01"类名的属性

这部分识别的是新闻列表区域的开始标记，即寻找一个具有class="hs01"的div元素。在新浪财经网页中，这个特定的class通常用于标记新闻列表的容器。

### 2. `.+?`
- `.` - 匹配任何字符（在使用re.S标志的情况下，包括换行符）
- `+?` - 表示前面的模式（任何字符）可以出现一次或多次，但采用非贪婪（最小）匹配

这部分会匹配第一个div和第二个div之间的所有内容，也就是整个新闻列表区域。

### 3. `<div\sclass="hs01">`
- 与第一部分相同，但在这里表示的是另一个相同类型div的开始，即下一个新闻列表区块的开始

### 工作原理
这个正则表达式使用了"包夹法"：它找到两个相同标记（`<div class="hs01">`）之间的所有内容。在新浪财经网页中，这两个标记之间通常包含了一组新闻项目。使用非贪婪匹配（`+?`）确保只匹配到第一个出现的第二个标记，不会过度匹配。

实际上，这个表达式会提取从第一个具有class="hs01"的div开始，到下一个具有相同class的div之前的所有HTML内容。这使得代码能够精确定位包含当前页面新闻列表的HTML块。

## 第二个正则表达式：p

```python
p = r'<li><a\shref="(.+?shtml).+?"\starget="_blank">(.+?)</a><span>\((.+?)\)</span></li>'
```

这个正则表达式的目的是从第一个表达式提取的HTML块中进一步解析出每条新闻的具体信息。让我详细分解它：

### 1. `<li>`
- 匹配列表项开始标签，表示一条新闻的开始

### 2. `<a\shref="(.+?shtml)`
- `<a` - 匹配超链接标签的开始
- `\s` - 匹配一个空白字符
- `href="` - 匹配href属性
- `(.+?shtml)` - 第一个捕获组，匹配以"shtml"结尾的URL
  - `.+?` - 非贪婪匹配任意字符
  - `shtml` - 匹配"shtml"字符串（新浪新闻链接的典型后缀）

这部分用于捕获新闻的URL链接，且只捕获到".shtml"为止。

### 3. `.+?"\starget="_blank">`
- `.+?"` - 非贪婪匹配href属性后的剩余部分直到引号结束
- `\s` - 匹配一个空白字符
- `target="_blank">` - 匹配target属性，这表示链接会在新窗口打开

这部分处理超链接标签的中间部分，但不捕获这些内容。

### 4. `(.+?)</a>`
- 第二个捕获组，匹配超链接的文本内容
- `(.+?)` - 非贪婪匹配任意字符（这是新闻的标题）
- `</a>` - 匹配超链接标签的结束

这部分用于捕获新闻的标题。

### 5. `<span>\((.+?)\)</span>`
- `<span>` - 匹配span标签的开始
- `\(` - 匹配左括号（需要转义因为括号在正则表达式中有特殊含义）
- `(.+?)` - 第三个捕获组，非贪婪匹配括号内的内容（这通常是新闻的发布时间）
- `\)` - 匹配右括号
- `</span>` - 匹配span标签的结束

这部分用于捕获新闻的发布时间，它通常包含在括号内的span标签中。

### 6. `</li>`
- 匹配列表项结束标签，表示一条新闻的结束

### 工作原理举例

假设有这样的HTML片段：
```html
<li><a href="https://finance.sina.com.cn/stock/stocknews/2023-05-21/doc-inews12345678.shtml" target="_blank">上市公司一季度业绩分析</a><span>(2023-05-21 09:45)</span></li>
```

应用此正则表达式后，会捕获3个组：
1. URL: `https://finance.sina.com.cn/stock/stocknews/2023-05-21/doc-inews12345678.shtml`
2. 标题: `上市公司一季度业绩分析`
3. 时间: `2023-05-21 09:45`

## 两个正则表达式的协同工作

在爬虫代码中，这两个正则表达式配合使用：
1. 首先，`pcheck`用于从整个网页中提取出包含新闻列表的HTML区块
2. 然后，`p`用于从这个区块中解析出每条新闻的链接、标题和发布时间

这种两阶段的提取方法有几个好处：
- 减少处理的HTML数量，提高效率
- 降低正则表达式复杂度，增加可维护性
- 减少误匹配的可能性，提高准确性

In [4]:
def myfun_crawl_sina_news():
    # 设置要爬取的新浪财经新闻页面URL（cid=56592代表特定财经版块）
    url = 'https://finance.sina.com.cn/roll/index.d.html?cid=56592&page='
    
    # 设置请求头，模拟Firefox浏览器访问，避免被网站识别为爬虫
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:137.0) Gecko/20100101 Firefox/137.0'}

    # 定义两个正则表达式：
    # pcheck用于检查和提取包含新闻列表的HTML部分
    pcheck = r'<div\sclass="hs01">.+?<div\sclass="hs01">'
    # p用于从新闻列表中提取每条新闻的链接、标题和时间
    p = r'<li><a\shref="(.+?shtml).+?"\starget="_blank">(.+?)</a><span>\((.+?)\)</span></li>'
    # 编译正则表达式以提高效率
    objp = re.compile(p, re.S)

    # 打开文件准备写入数据
    fw = open('Data_SinaNews.txt', 'w')
    
    # 只爬取第一页（注意这里的循环范围是1到2，实际只执行一次）
    for i in range(1, 2):
        # 使用无限循环，直到成功获取数据
        while True:
            try:
                # 发送GET请求获取网页内容，设置10秒超时
                res = requests.get(url+str(i), headers=headers, timeout=10)
                # 设置响应编码为utf-8
                res.encoding = 'utf-8'
                # 获取HTML内容
                html = res.text
                # 检查是否包含新闻列表（通过正则表达式查找）
                mcheck = re.search(pcheck, html, re.S)
                # 如果找到新闻列表，则提取该部分并跳出循环
                if len(mcheck.group()) > 0:
                    html = mcheck.group()
                    break
            except:
                # 如果请求超时，打印错误信息
                print('failing to crawl the data because of timeout')
            # 随机等待10到30秒，避免频繁请求被封IP
            time.sleep(np.random.randint(10, 30))
        
        # 从提取的HTML中找出所有新闻条目
        match = objp.findall(html)
        # 遍历每条新闻，提取链接、标题和时间
        for line in match:
            # 以制表符分隔写入文件
            fw.write('{:s}\t{:s}\t{:s}\n'.format(line[0], line[1], line[2]))
    # 关闭文件
    fw.close()

# 调用函数执行爬虫
myfun_crawl_sina_news()

In [5]:
# 发送GET请求获取股票数据，访问新浪财经提供的JSON API
# symbol=sh601633表示上海证券交易所的601633股票
# scale=240表示240分钟（即4小时）K线数据
# ma=no表示不需要移动平均线数据
# datalen=10000表示获取最多10000条数据
res = requests.get('http://money.finance.sina.com.cn/quotes_service/api/json_v2.php/CN_MarketData.getKLineData?symbol=sh601633&scale=240&ma=no&datalen=10000')

# 将响应转换为JSON对象
data_json = res.json() 

# 将数据写入文本文件的代码
fw = open('data_sina_api.txt', 'w')
fw.write('day, open, high, low, close, volume\n')
for i in range(len(data_json)):
    dj = data_json[i]
    fw.write('{:s},{:s},{:s},{:s},{:s},{:s}\n'.format(dj['day'], dj['open'], dj['high'],
                                                    dj['low'], dj['close'], dj['volume']))
fw.close()

# 使用pandas将JSON数据转换为DataFrame格式，便于数据分析
stock_data = pd.DataFrame(data_json)
# 打印股票数据
print(stock_data)

             day    open    high     low   close    volume
0     2011-09-28  12.800  12.800  11.810  11.850  68260880
1     2011-09-29  11.600  12.700  11.510  12.380  55975244
2     2011-09-30  12.280  12.680  12.020  12.460  38487768
3     2011-10-10  12.460  12.470  11.210  11.320  34180024
4     2011-10-11  11.600  12.450  11.530  12.330  55751612
...          ...     ...     ...     ...     ...       ...
3258  2025-04-10  23.460  23.600  23.130  23.300  28289721
3259  2025-04-11  23.300  23.450  23.070  23.330  19653339
3260  2025-04-14  23.700  23.800  23.420  23.590  19166601
3261  2025-04-15  23.600  23.610  23.080  23.250  18755445
3262  2025-04-16  23.320  23.340  22.870  23.340  17145150

[3263 rows x 6 columns]
